# My Solution for Siraj's defi 

My solution is generic and works with any number of layers, and any number of nodes in each layer.

Take care of the difference of numpy dot and * !

In [1]:
from numpy import exp, array, random, dot, matmul

In [2]:
A = array([[0, 1], [1, 0]])
B = array([[0, 1]]).T
C = array([[3, 1], [2, 1]])
print "A"
print A
print "B"
print B
print "C"
print C
print "A * B"
print A * B
print "A.dot(B)"
print A.dot(B)
print "The solution is different because * is the Element-wise Multiplication and dot is matrice multiplication"
print "A * C"
print A * C
print "A.dot(C)"
print A.dot(C)
print "np.matmul(A, C)"
print matmul(A, C)
print "The result of dot and matmul are the same if and only if the matrices are two dimensional"

A
[[0 1]
 [1 0]]
B
[[0]
 [1]]
C
[[3 1]
 [2 1]]
A * B
[[0 0]
 [1 0]]
A.dot(B)
[[1]
 [0]]
The solution is different because * is the Element-wise Multiplication and dot is matrice multiplication
A * C
[[0 1]
 [2 0]]
A.dot(C)
[[2 1]
 [3 1]]
np.matmul(A, C)
[[2 1]
 [3 1]]
The result of dot and matmul are the same if and only if the matrices are two dimensional


In [3]:
class NeuralNetwork():
    
    # Add variable for define the layers. Each layer is a tuple (nb_inputs, nb_neurons)
    def __init__(self, *layers):
        # Seed the random number generator, so it generates the same numbers
        # every time the program runs.
        random.seed(1)

        # We parse the array of tuples and initializes the weigths matrix for each layers
        self.layer_weights = []
        for layer in layers:
            self.layer_weights.append(2 * random.random((layer[0], layer[1])) - 1)
            
        self.nb_layer = len(layers)

    # The Sigmoid function, which describes an S shaped curve.
    # We pass the weighted sum of the inputs through this function to
    # normalise them between 0 and 1.
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    # It indicates how confident we are about the existing weight.
    def __sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def printLog(self, num_layer, layer_error, delta, layer_adjustment):
        print "Layer ", num_layer ,"error"
        print layer_error
        print "Layer ", num_layer ,"delta"
        print delta
        print "layer ", num_layer ,"adjustment"
        print layer_adjustment

    # We train the neural network through a process of trial and error.
    # Adjusting the synaptic weights each time.
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in xrange(number_of_training_iterations):
            
            print "====== Forward Propagation ======"
            outputs = self.think(training_set_inputs)

            print "====== Backward Propagation ======"
            
            layer_delta = list()
            layer_adjustments = list()
            
            # Calculate the error from the last layer which is based on the output of the Forward Propagation
            # (The difference between the desired output and the predicted output).
            layer_error = training_set_outputs - outputs[-1]

            # Multiply the error by the gradient of the Sigmoid curve of the last 
            # layer output (output computed by the FP)
            delta = layer_error * self.__sigmoid_derivative(outputs[-1])
            layer_delta.append(delta)

            # Compute the adjustment : multiply the (layer -1) by the delta
            layer_adjustment = outputs[-2].T.dot(delta)
            layer_adjustments.append(layer_adjustment)
            
            self.printLog(self.nb_layer, layer_error, delta, layer_adjustment)

            for index in range(self.nb_layer -1):  # 0 to n-1 (n = output_layer already processed)

                num_layer = (self.nb_layer - index) -1
            
                last_delta = layer_delta[-1]

                # Calculate the error : delta error next layer * weight next layer
                # layer1_error = layer2_delta.dot(self.layer2.synaptic_weights.T)
                layer_error = last_delta.dot(self.layer_weights[num_layer].T)

                # Compute the delta backprop error term : error * derivate of the next layer
                # layer1_delta = layer1_error * self.__sigmoid_derivative(output_from_layer_1)
                delta = layer_error * self.__sigmoid_derivative(outputs[num_layer-1])
                layer_delta.append(delta)

                # Compute the adjustment
                # layer1_adjustment = training_set_inputs.T.dot(layer1_delta)
                if (num_layer >= 2) :
                    layer_adjustment = outputs[num_layer-2].T.dot(delta)
                else :
                    layer_adjustment = training_set_inputs.T.dot(delta)
                layer_adjustments.append(layer_adjustment)
                
                self.printLog(num_layer, layer_error, delta, layer_adjustment)

            # Adjust the weights.
            # The two lists are stored in reverse order
            for j in range(self.nb_layer):
                self.layer_weights[j] += layer_adjustments[-(j+1)]

    # The neural network thinks.
    # The first layer is computing with the inputs so it's why the calcul is not in the loop
    def think(self, inputs):
        
        # This list stores all the computes for each layer
        layer_back_prop_value = list()
        
        # Compute the first layer and initialize the variable layer_back_prop_value
        layer_back_prop_value.append(self.__sigmoid(dot(inputs, self.layer_weights[0])))
        
        print "Layer 0"
        print layer_back_prop_value[-1]
            
        # Compute each layer using the value of the previous layer
        for index in range(1,self.nb_layer):
            back_prop_value = self.__sigmoid(dot(layer_back_prop_value[-1], self.layer_weights[index]))
            layer_back_prop_value.append(back_prop_value)
            
            print "Layer ", index
            print back_prop_value

        print " "
        return layer_back_prop_value
    

In [4]:
# Initialise a multi layer neural network. (3 layers)
# Each layer is represented by a tuple (nb_inputs, nb_neurons)
neural_network = NeuralNetwork((3, 4), (4, 4), (4, 1))

print("======Random starting Layer weights: ======")

for i in range(neural_network.nb_layer):
    print "Layer ", i
    print neural_network.layer_weights[i]
print " "
            
# The training set. We have 4 examples, each consisting of 3 input values
# and 1 output value.
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_set_outputs = array([[0, 1, 1, 0]]).T


# Train the neural network using a training set.
# Do it 10,000 times and make small adjustments each time.
neural_network.train(training_set_inputs, training_set_outputs, 1) #10000

print("====== New weights after training: ======")
for i in range(neural_network.nb_layer):
    print "Layer ", i+1
    print neural_network.layer_weights[i]
print " "

# Test the neural network with a new situation.
print ("Considering new situation [1, 1, 0] -> ?: ")
res = neural_network.think(array([1, 1, 0]))
print res[-1]


======Random starting Layer weights: ======
Layer  0
[[-0.16595599  0.44064899 -0.99977125 -0.39533485]
 [-0.70648822 -0.81532281 -0.62747958 -0.30887855]
 [-0.20646505  0.07763347 -0.16161097  0.370439  ]]
Layer  1
[[-0.5910955   0.75623487 -0.94522481  0.34093502]
 [-0.1653904   0.11737966 -0.71922612 -0.60379702]
 [ 0.60148914  0.93652315 -0.37315164  0.38464523]
 [ 0.7527783   0.78921333 -0.82991158 -0.92189043]]
Layer  2
[[-0.66033916]
 [ 0.75628501]
 [-0.80330633]
 [-0.15778475]]
 
====== Forward Propagation ======
Layer 0
[[ 0.44856632  0.51939863  0.45968497  0.59156505]
 [ 0.25371248  0.42628115  0.14321233  0.41732254]
 [ 0.40795614  0.62674606  0.23841622  0.49377636]
 [ 0.28639589  0.32350963  0.31236398  0.51538526]]
Layer  1
[[ 0.59164352  0.78542488  0.18846121  0.37069838]
 [ 0.54483448  0.66937976  0.27965197  0.3774167 ]
 [ 0.5424749   0.73008524  0.20831066  0.35369355]
 [ 0.58735983  0.72188698  0.25967308  0.38872586]]
Layer  2
[[ 0.49835927]
 [ 0.46562171]
 [ 0.49